In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

a = llm.predict("행성은 몇개가 있나요?")  # davinci
b = chat.predict("행성은 몇개가 있나요?") # gpt-3.5-turbo

a,b


c:\STUDY\Programing\Python\gpt\3\env\lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\STUDY\Programing\Python\gpt\3\env\lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


('태양계에는 8개의 행성이 있습니다. 이들은 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 넵튠으로 이루어져 있습니다. 또한, 명왕성과 기암(haumea), 마케마케(haumea),  에리스( eris)와 같은 왜행성이 있지만, 현재까지는 왜행성을 행성으로 인정하지는 않았습니다.',
 '태양계에는 8개의 행성이 있습니다. 이들은 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성입니다. 추가로 몇몇 왜행성도 존재하지만, 공식적으로는 행성으로 분류되지 않습니다.')

In [2]:
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1 # 창의성을 나타내는 지표. 높을수록 창의적
)

from langchain.schema import HumanMessage, AIMessage, SystemMessage
# AIMessage : AI에 의해 보내지는거
# SystemMessage : 우리가 LLM에 설정들을 제공하기 위한 Message

messages = [
    SystemMessage(content="당신은 지리학자입니다. 당신은 오직 이탈리아어로 대답합니다."),
    # SystemMessage(content="you are a geography expert. And you only reply in italian."),
    AIMessage(content="ciao, mi chiamo Paolo"),
    HumanMessage(content="멕시코와 태국 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 무엇인가요?")
]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, ma 저는 이탈리아어로만 대답할 수 있습니다. 멕시코와 태국 사이의 거리는 대략 15,000km 정도입니다. 제 이름은 파올로입니다.')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
# ChatPromptTemplate : template을 message로 부터 만든다.
# PromptTemplate : string 으로 template을 만든다.

from langchain.callbacks import StreamingStdOutCallbackHandler # 답변 생성하는 과정 볼 수 있게

template = PromptTemplate.from_template("{국가1}과 {국가2}의 거리가 어떻게 되나요?")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0.1 # 창의성을 나타내는 지표. 높을수록 창의적
)

prompt = template.format(국가1='대한민국', 국가2='미국')

chat.predict(prompt)

'대한민국과 미국 사이의 거리는 직선거리로 약 10,000km 정도입니다. 하지만 실제로 이동할 때에는 비행기나 배 등을 이용하여 이동해야 하므로 실제 이동 거리는 더 길어질 수 있습니다. 대한민국과 미국은 서로 대륙이 다르기 때문에 이동 수단에 따라 거리가 달라질 수 있습니다.'

In [4]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다. 당신은 오직 {언어}로 대답합니다."),
    ("ai", "ciao, mi chiamo {이름}"),
    ("human", "{국가1}과 {국가2}의 거리가 어떻게 되나요? 그리고 당신의 이름은 무엇인가요?")
])

prompt = template.format_messages(
    언어 = "italian",
    이름 = "pauolo",
    국가1 = "대한민국",
    국가2 = "미국"
)

chat.predict_messages(prompt)

AIMessage(content='Mi dispiace, non posso fornire informazioni precise sulla distanza tra la Corea del Sud e gli Stati Uniti in questo momento. Il mio nome è Paolo. Posso aiutarti con altre domande sulla geografia?')

In [5]:
# 항상 텍스트로 대답하는 llm의 대답을 원하는 형태로 만들어보자
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello, world")

['Hello', 'world']

In [6]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=10, 
    question="색상이 뭐가 있을까?"
)

result = chat.predict_messages(prompt)
result

AIMessage(content='빨강, 주황, 노랑, 초록, 파랑, 남색, 보라, 핑크, 갈색, 회색')

In [7]:
p = CommaOutputParser()
p.parse(result.content)

['빨강', '주황', '노랑', '초록', '파랑', '남색', '보라', '핑크', '갈색', '회색']

In [8]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":5,
    "question":"포켓몬 종류 말해줘"
})

['피카츄', '파이리', '꼬부기', '이상해씨', '꼬마돌']

In [9]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.callbacks.manager import CallbackManager

# 스트리밍 콜백 핸들러 생성
streaming_handler = StreamingStdOutCallbackHandler()

# 콜백 매니저 생성
callback_manager = CallbackManager([streaming_handler])

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    callback_manager=callback_manager,
    streaming=True
)

# 첫 번째 쉐프 프롬프트
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 월드클래스 국제 요리사입니다. 당신은 어떤 종류의 요리든 쉽게 구할 수 있는 재료로 따라하기 쉬운 레시피를 만들어 줍니다."),
    ("human", "나는 {cuisine}를 요리하기 원합니다.")
])

# 채식주의자 쉐프 프롬프트
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 채식주의자를 위한 세프입니다. 전통적인 채식주의자용 레시피에 특화되어 있습니다. 당신은 대체 재료를 찾고, 준비하는 방법에 대해 설명합니다. 기존 레시피를 너무 많이 변경해서는 안됩니다. 만약 다른 대체품이 없다면 그냥 레시피를 모른다고 말하세요"),
    ("human", "{recipe}")
])

# 체인 구성
chef_chain = chef_prompt | chat | StrOutputParser()
veg_chain = veg_chef_prompt | chat | StrOutputParser()

# 최종 체인 구성
final_chain = {"recipe": chef_chain} | veg_chain

# 체인 실행
print("일반 요리사의 레시피:")
result = final_chain.invoke({"cuisine": "indian"})

print("\n채식주의자 요리사의 레시피:")
print(result)

일반 요리사의 레시피:
인도 요리를 만들기 위해 가장 기본적인 재료들은 고추, 생강, 마늘, 양파, 토마토, 고추가루, 코리앤더, 큐민, 고추, 우유 또는 요거트 등이 있습니다. 이러한 재료들을 이용하여 맛있는 치킨 커리를 만들어 보는 것은 어떨까요?

인도식 치킨 커리 레시피:

재료:
- 닭가슴살 500g
- 양파 1개
- 마늘 4쪽
- 생강 1조각
- 토마토 2개
- 요거트 1컵
- 코리앤더 가루 1큰술
- 큐민 가루 1큰술
- 고추 가루 1큰술
- 소금
- 후추
- 식용유

요리과정:
1. 양파, 마늘, 생강을 다져줍니다.
2. 닭가슴살을 한입 크기로 잘라 소금과 후추로 조리고 냉장고에 30분간 숙성시킵니다.
3. 팬에 식용유를 두르고 양파, 마늘, 생강을 볶아 향을 낸 후 다져놓은 토마토를 넣어 볶아줍니다.
4. 코리앤더 가루, 큐민 가루, 고추 가루를 넣고 잘 섞어줍니다.
5. 숙성시킨 닭가슴살을 넣고 익힌 후 요거트를 넣어 끓여줍니다.
6. 소금과 후추로 간을 맞추고 적당한 두께가 될 때까지 끓여줍니다.
7. 밥이나 나안, 로티와 함께 내놓아 맛있게 즐기세요.

이렇게 간단한 재료로도 인도식 치킨 커리를 만들어 낼 수 있습니다. 맛있게 요리하세요!채식주의자를 위한 대체재료로는 대체로 식물성 식품을 사용해야 합니다. 인도식 치킨 커리 레시피에서는 요거트가 주된 대체가 어려운 재료입니다. 대체재료로는 코코넛 밀크나 코코넛 요거트를 사용할 수 있습니다. 코코넛 밀크는 요거트와 비슷한 크리미한 맛을 줄 수 있습니다. 또한 콩 요거트나 콩 크림도 대체재료로 사용할 수 있습니다. 이러한 대체재료들을 사용하여 인도식 치킨 커리를 만들어보세요.
채식주의자 요리사의 레시피:
채식주의자를 위한 대체재료로는 대체로 식물성 식품을 사용해야 합니다. 인도식 치킨 커리 레시피에서는 요거트가 주된 대체가 어려운 재료입니다. 대체재료로는 코코넛 밀크나 코코넛 요거트를 사용할 수 있습니다. 코코넛 밀크는 요거트와 비슷한 크리미한 맛을 줄 수 있습니다. 또한 콩 요거트나 콩 크림

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# t = PromptTemplate(
#     template="{국가}의 수도는 어디인가요?",
#     input_variables=["country"]
# )

# t.format(국가="미국")

chat.predict("프랑스에 대해 무엇을 알고 있어?")

프랑스는 유럽 대륙 서부에 위치한 나라로 파리를 수도로 하고 있습니다. 프랑스는 세계적으로 유명한 문화와 예술의 중심지로서 유명하며, 에펠탑, 루브르 박물관, 베르사유 궁전 등 많은 유명한 관광지가 있습니다. 또한 프랑스는 세계적으로 유명한 요리와 와인으로도 유명하며, 프랑스어가 공용어로 사용되고 있습니다. 프랑스는 유럽 연합의 주요 회원국 중 하나이며, 세계 경제에서도 중요한 역할을 하고 있습니다.

'프랑스는 유럽 대륙 서부에 위치한 나라로 파리를 수도로 하고 있습니다. 프랑스는 세계적으로 유명한 문화와 예술의 중심지로서 유명하며, 에펠탑, 루브르 박물관, 베르사유 궁전 등 많은 유명한 관광지가 있습니다. 또한 프랑스는 세계적으로 유명한 요리와 와인으로도 유명하며, 프랑스어가 공용어로 사용되고 있습니다. 프랑스는 유럽 연합의 주요 회원국 중 하나이며, 세계 경제에서도 중요한 역할을 하고 있습니다.'

In [14]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_template="""
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: {country}에 대해 무엇을 알고 있어?",
    input_variables=["country"]
)

prompt.format(country="프랑스")

'\n    Human: What do you know about France?\n    AI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Italy?\n    AI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\n\n\n    Human: What do you know about Greece?\n    AI: \n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\n\nHuman: 프랑스에 대해 무엇을 알고 있어?'

In [15]:
chain = prompt | chat

chain.invoke({
    "country":"미국"
})

AI: 
미국에 대해 알고 있는 정보는 다음과 같습니다:
수도: 워싱턴 D.C.
언어: 영어
음식: 핫도그, 햄버거
통화: 미국 달러

AIMessageChunk(content='AI: \n미국에 대해 알고 있는 정보는 다음과 같습니다:\n수도: 워싱턴 D.C.\n언어: 영어\n음식: 핫도그, 햄버거\n통화: 미국 달러')

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "you are a geography expert, you give short answers"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country":"미국"
})

미국 is the Korean name for the United States. Here is some information:
        Capital: Washington D.C.
        Language: English
        Food: Hamburgers and Hotdogs
        Currency: US Dollar

AIMessageChunk(content='미국 is the Korean name for the United States. Here is some information:\n        Capital: Washington D.C.\n        Language: English\n        Food: Hamburgers and Hotdogs\n        Currency: US Dollar')